In [ ]:
import sys
import os

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy
import pandas as pd
import datetime
import copy

import matplotlib.dates as mdates
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from matplotlib import ticker as mticker


# Local files
__root__ = '../../'
__pydirs__ = ['py', 'settings']
module_path = os.path.abspath(os.path.join(__root__[:-1]))
if module_path not in sys.path:
    for pydir in __pydirs__:
        sys.path.append(f"{module_path}/{pydir}/")

import T23_setup

import log_ret

import misc_functions as misc_fns
import plot_functions as plot_fns
import dist_functions as dist_fns
import hawkes_functions as hwks_fns
import log_ret_functions as lr_fns
import cvol_functions as cvol_fns
import greek_roman as GR

In [ ]:
make_plot = True

In [ ]:
plot_fns.plt_setup(plt, SMALL_SIZE=6.5, MEDIUM_SIZE=6.5, BIGGER_SIZE=8)
cols = plot_fns.calc_cols()
cols_CI = plot_fns.calc_cols_CI()
cols_tail = plot_fns.calc_cols_tail(rows=True)

In [ ]:
r_r = f"r"

resid_r = f"\\tilde{{{r_r}}}"
sim_r = f"\\acute{{{r_r}}}"

tilde_t_cont = "\\Lambda"
tilde_t_disc = "\\Pi"

In [ ]:
Tmisc = T23_setup.T23_misc()
fstmp = T23_setup.T23_Cfstmp(root=__root__, n=4, title='T23')
fstmp.calc_fnsubs_default()

In [ ]:
fix_a_lambda = True
tick='SPX'
_setup_T23 = T23_setup.setup_T23(__C__='Cm4', __fit__=False, __vares__=False,
                                    set_cv=False, set_tt=True,
                                    tt_fix_a_lambda=fix_a_lambda,
                                    all_end=pd.Timestamp('2023-09-16'),
                                    )

In [ ]:
_setup_T23['tt_a_u']

In [ ]:
B20 = log_ret.log_ret(dir_base=__root__, ticks=_setup_T23['ticks'])
B20.set_tk(**misc_fns.dict_key_rm(_setup_T23, keys_rm='ticks'))

In [ ]:
cv_mp_tick_time = B20.set_cv_mp_tick(set_am_stats=True, set_eval=False, append_fit_cv_w=True, append_vrs_cv_w=False, write_tick=False)

In [ ]:
read_ticks = ['SPX', 'DJI', 'DAX', 'CAC', 'NKX', 'HSI']
results_dir = lr_fns.calc_dir_suff(suff='res', dir_base=__root__, pref_base=True, use_defaults=True)

periods_list = [B20.tk[read_ticks[0]].pd.get_dates_str(dates_format='t0_t1/_a1')]

mod_dict = {fn: {mod: None for mod in ['cv', 'cv_evt', 'tt_hb']} for fn in ['fit', 'vares']}


suf_dict = copy.deepcopy(mod_dict)
for mod in suf_dict['fit']:
    suf_dict['fit'][mod] = {'train': {suf: None for suf in ['est', 'se']}}
suf_dict['fit']['cv_evt'] = {'train': {suf: None for suf in ['est']}}
#suf_dict['fit'][f'tt']['train']['_meta'] = None


for mod in suf_dict['vares']:
    suf_dict['vares'][mod] = {'forecast': {suf: None for suf in ['vrs']}}


In [ ]:
B20rA = {fn: {mod: {sec: {suffix: pd.DataFrame() for suffix in suf_dict[fn][mod][sec]} for sec in suf_dict[fn][mod]} for mod in suf_dict[fn]} for fn in suf_dict}
B20rT = {tick: {period: copy.deepcopy(B20rA) for period in periods_list} for tick in read_ticks}

for tick in B20rT:
    for period in B20rT[tick]:
        for fn in B20rT[tick][period]:
            for mod in B20rT[tick][period][fn]:
                for sec in B20rT[tick][period][fn][mod]:
                    if sec=='train':
                        p_str = period[:21]
                    else:
                        p_str = period
                    t_dir = f'{results_dir}{fn}/{tick}/{p_str}/'
                    for suffix in B20rT[tick][period][fn][mod][sec]:
                        f_name = f'{fn}_{mod}_{suffix}.csv'
                        B20rT[tick][period][fn][mod][sec][suffix] = pd.read_csv(f'{t_dir}{f_name}', sep=',')
                        B20rA[fn][mod][sec][suffix] = B20rA[fn][mod][sec][suffix].append(B20rT[tick][period][fn][mod][sec][suffix])


In [ ]:
vrs_cols =  {
                'tick': ['tick'],
                'train_dates': ['train_start', 'train_end'],
                'all_dates': ['train_start', 'train_end', 'all_end'],
                'cv': ['vol', 'p', 'o', 'q', 'dist'],
                'tt': ['a_u_1', 'fix_a_lambda', 'mode', 'mark'],
                'tt_hb': ['dist', 'bound'],
                'vrs_tail': ['tail', 'a_q', 'period'],
                'vrs_a_q': ['Q_T', 'Q_T_1' 'Q_T_1_a_q'],
                'vrs_CT_Q_p_C': ['CT_Q_binom_p', 'CT_Q_UC_p', 'CT_Q_CC_p'],
                'vrs_CT_Q_p_D': np.concatenate([['CT_Q_DL_p'], [f'CT_Q_DQ_{i}_p' for i in range(3)]]),
                'vrs_CT_E_ZMD': ['CT_E_ZMD_t', 'CT_E_ZMD_b_opt_CB'],
                'vrs_CT_E_ZMD_F': ['CT_E_ZMD_1000_0:0_F'],
                'vrs_S': ['S_Q_G', 'S_E_AL', 'S_E_NZ', 'S_E_FZG'],
            }

In [ ]:
# set the colormap and centre the colorbar
class MidpointNormalize(matplotlib.colors.Normalize):
	"""
	Normalise the colorbar so that diverging bars work there way either side from a prescribed midpoint value)

	e.g. im=ax1.imshow(array, norm=MidpointNormalize(midpoint=0.,vmin=-100, vmax=100))
	"""
	def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
		self.midpoint = midpoint
		matplotlib.colors.Normalize.__init__(self, vmin, vmax, clip)

	def __call__(self, value, clip=None):
		# I'm ignoring masked values and all kinds of edge cases to make a
		# simple example...
		x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
		return np.ma.masked_array(np.interp(value, x, y), np.isnan(value))

# set the colormap and centre the colorbar
class MultipointNormalize(matplotlib.colors.Normalize):
	"""
	Normalise the colorbar so that diverging bars work there way either side from a prescribed midpoint value)

	e.g. im=ax1.imshow(array, norm=MidpointNormalize(midpoint=0.,vmin=-100, vmax=100))
	"""
	def __init__(self, vmin=None, vmax=None,  vpoints=None, points=None, clip=False):

		if vpoints is None:
			vpoints=0.5
		if points is None:
			points=0.5
		self.vpoints = misc_fns.make_iterable_array(vpoints)
		self.points = misc_fns.make_iterable_array(points)
		matplotlib.colors.Normalize.__init__(self, vmin, vmax, clip)
		self.x = np.concatenate([[self.vmin], self.vpoints, [self.vmax]])
		self.y = np.concatenate([[0], self.points, [1]])


	def __call__(self, value, clip=None):
		# I'm ignoring masked values and all kinds of edge cases to make a
		# simple example...
		return np.ma.masked_array(np.interp(value, self.x, self.y), np.isnan(value))

In [ ]:
# create an array of random vlues - you might read in a raster dataset
x=25
y=25
ras=np.random.randint(-1000,3000,size=(x*y)).reshape(x,y)
cmap=matplotlib.cm.RdBu # set the colormap to soemthing diverging
plt.imshow(ras, cmap=cmap), plt.colorbar(), plt.show()

In [ ]:
elev_min=-1000
elev_max=3000
mid_val=0

plt.imshow(ras, cmap=cmap, clim=(elev_min, elev_max), norm=MidpointNormalize(midpoint=mid_val,vmin=elev_min, vmax=elev_max))
plt.colorbar()
plt.show()

In [ ]:
elev_min=-1000
elev_max=3000
mid_val=0

plt.imshow(ras, cmap=cmap, clim=(elev_min, elev_max), norm=MultipointNormalize(vpoints=mid_val,vmin=elev_min, vmax=elev_max))
plt.colorbar()
plt.show()

In [ ]:
elev_min=-1000
elev_max=3000
mid_val=0

plt.imshow(ras, cmap=cmap, clim=(elev_min, elev_max), norm=MultipointNormalize(vpoints=[-900,2900], 
                                                        points=[1/4,3/4], 
                                                        #points=[1/3,2/3], 
                                                        vmin=elev_min, vmax=elev_max))
plt.colorbar()
plt.show()

In [ ]:
def im6_plot_setup(name="fig", nrows=3, ncols=2, pb_ratio=3):

    fig = {
                "name": name,
                "fig": None,
                "gs": None,
                "ax":   [
                            [None for j in range(ncols)] for i in range(nrows)
                        ],
                "bar": None,
            }
    fig["fig"] = plt.figure(constrained_layout=True)
    fig["gs"] = fig["fig"].add_gridspec(nrows, 1+pb_ratio*ncols)
    for i in range(len(fig["ax"])):
        for j in range(len(fig["ax"][i])):
            fig["ax"][i][j] = fig["fig"].add_subplot(fig["gs"][i, pb_ratio*j:pb_ratio*(1+j)])
    fig["bar"] = fig["fig"].add_subplot(fig["gs"][:, -1])

    for i in range(len(fig["ax"])):
        for j in range(len(fig["ax"][i])):
            fig["ax"][i][j].tick_params(
                                            which='both',      
                                            bottom=True, labelbottom=False,     
                                            top=True, labeltop=False,
                                            left=True, labelleft=False,     
                                            right=True, labelright=False,         
                                            direction="out",
                                        )
            if len(fig["ax"]) > 1:
                fig["ax"][0][j].xaxis.set_label_position("top")
                fig["ax"][0][j].tick_params(labeltop=True)
            fig["ax"][-1][j].tick_params(labelbottom=True)
        if len(fig["ax"][i]) > 1:
            fig["ax"][i][-1].yaxis.set_label_position("right")
            fig["ax"][i][-1].tick_params(labelright=True)
        fig["ax"][i][0].tick_params(labelleft=True)
    
    fig["bar"].tick_params(
                                which='both',      
                                bottom=False, labelbottom=False,     
                                top=False, labeltop=False,
                                left=True, labelleft=False,     
                                right=True, labelright=True,         
                                direction="out",
                            )
    fig["bar"].yaxis.set_label_position('right')

    fig["fig"].set_size_inches(5.7, 6)
    fig["fig"].tight_layout()

    return fig

In [ ]:
fig_test = im6_plot_setup(name="fig", nrows=2, ncols=2, pb_ratio=4)

fig_test["fig"].set_size_inches(8.5, 4)

In [ ]:
fig_test = im6_plot_setup(name="fig", nrows=2, ncols=2, pb_ratio=4)

fig_test["fig"].set_size_inches(5.7, 6)

In [ ]:
def im4_plot_setup(name="fig", nrows=2, ncols=2, pb_ratio=5):

    fig = {
                "name": name,
                "fig": None,
                "gs": None,
                "ax":   [
                            [None for j in range(ncols)] for i in range(nrows)
                        ],
                "bar": None,
            }
    fig["fig"] = plt.figure(constrained_layout=True)
    fig["gs"] = fig["fig"].add_gridspec(1+pb_ratio*nrows, ncols)
    for i in range(len(fig["ax"])):
        for j in range(len(fig["ax"][i])):
            fig["ax"][i][j] = fig["fig"].add_subplot(fig["gs"][pb_ratio*i:pb_ratio*(1+i), j])
    fig["bar"] = fig["fig"].add_subplot(fig["gs"][-1, :])

    for i in range(len(fig["ax"])):
        for j in range(len(fig["ax"][i])):
            fig["ax"][i][j].tick_params(
                                            which='both',      
                                            bottom=True, labelbottom=False,     
                                            top=True, labeltop=False,
                                            left=True, labelleft=False,     
                                            right=True, labelright=False,         
                                            direction="out",
                                        )
            if len(fig["ax"]) > 1:
                fig["ax"][0][j].xaxis.set_label_position("top")
                fig["ax"][0][j].tick_params(labeltop=True)
            fig["ax"][-1][j].tick_params(labelbottom=True)
        if len(fig["ax"][i]) > 1:
            fig["ax"][i][-1].yaxis.set_label_position("right")
            fig["ax"][i][-1].tick_params(labelright=True)
        fig["ax"][i][0].tick_params(labelleft=True)
    
    fig["bar"].tick_params(
                                which='both',      
                                bottom=True, labelbottom=True,     
                                top=True, labeltop=False,
                                left=False, labelleft=False,     
                                right=False, labelright=False,         
                                direction="out",
                            )
    fig["bar"].yaxis.set_label_position('right')

    fig["fig"].set_size_inches(5.7, 8)
    fig["fig"].tight_layout()

    return fig

In [ ]:
fig_test = im4_plot_setup(name="fig", nrows=2, ncols=2, pb_ratio=4)

fig_test["fig"].set_size_inches(5.7, 6)

In [ ]:
def set_csv(csv_path):
    
    csv = pd.read_csv(csv_path, sep=',', index_col='Date')
    csv.index = np.array([pd.Timestamp(csv.index[k]) for k in range(csv.index.size)])
    return csv

def calc_tick_std(ticks, periods_str=None, train_end=None, all_end=None):

    if periods_str is not None:
        train_end = periods_str[11:21]
        all_end = periods_str[22:32]


    tick_std = {tick: None for tick in ticks}


    for tick in ticks:
        tick_dir = f"{__root__}data/^{tick.lower()}_d.csv"
        csv = set_csv(tick_dir)

        lr = np.log(csv['Close']).diff()[1:]
        lr_f = lr.iloc[np.logical_and(lr.index >= pd.Timestamp(train_end),
                                                                lr.index < pd.Timestamp(all_end))]

        tick_std[tick] = np.std(lr_f.values)

    return tick_std

ticks = np.concatenate([['SPX', 'DJI'],
        ['DAX', 'CAC'], 
        ['NKX', 'HSI']])

tick_std = calc_tick_std(ticks=ticks, periods_str=periods_list[-1].replace('/', ''))
tick_std

In [ ]:
ticks = np.concatenate([['SPX', 'DJI'],
        ['DAX', 'CAC'], 
        ['NKX', 'HSI']])
date_head = ['train_start', 'train_end', 'all_end']
tails = ['left', 'right']
flip_right=False

periods = ['forecast']

tt_fix_a_lambdas = [True]


#hb_dists=['norm', 't']
hb_dists=['t']

#modes = ['bi', 'ci']
modes = ['ci']

#one_tails= ['s', 'd']
one_tails= ['s']

#o_evt = [1,0]
o_evt = [1]


dist_symbol = {
                'norm': f"\\mathcal{{N}}",
                't': f"\\mathcal{{S}}",
            }

pdict = {mod: dict() for mod in ['G', 'Ge', 'H']}

pdict['H'] = {
                'mode': ['ci_s', modes[0]],
                'mark': ['mark' for k in range(2)],
                'bound':    [['both' for k in range(2)] for l in range(2)],
                'model':   [f"1", f"2"],     
            }

pdict['H']['N']=len(pdict['H']['model'])


base_G = {
            'vol': ['GARCH', 'GARCH', 'GARCH'],
            'dist': ['norm', 't', 't'],
            'o': [0,0,1],
            'label': [
                        cvol_fns.calc_cv_symb(vol='GARCH', o=0, dist='norm', a_u=0, show_a_u_val=True),
                        cvol_fns.calc_cv_symb(vol='GARCH', o=0, dist='t', a_u=0, show_a_u_val=True),
                        cvol_fns.calc_cv_symb(vol='GARCH', o=1, dist='t', a_u=0, show_a_u_val=True),
                    ],
        }


base_G['N']=len(base_G['vol'])
base_G['L']=6

base_G['o_str'] = ''
base_G['dist_str'] = ''
for g in range(len(base_G['vol'])):
    base_G['o_str'] += f"{base_G['o'][g]},"
    base_G['dist_str'] += f"{dist_symbol[base_G['dist'][g]]},"

flip_G = copy.deepcopy(base_G)
for item in ['vol', 'dist', 'o', 'label']:
    flip_G[item] = np.flip(flip_G[item])


pdict['G'] = copy.deepcopy(base_G)

pdict['Ge'] = {
            'vol': ['GARCH'],
            'dist': [None],
            'o': [o_evt[0]],
            'label': [cvol_fns.calc_cv_symb(vol='GARCH', o=o_evt[0], dist='norm', show_a_u_symb=True)],
        }
pdict['Ge']['N']=len(pdict['Ge']['vol'])


a_lambda_tick = [0.08, 0.16]
a_lambda_tick_Ge = a_lambda_tick

a_u_ticks = 0.02*np.arange(1,8)

G_start=1
fs=6.5

max_i = 2


In [ ]:
tests = ['UC']
#tests = ['UC']


#cmap = matplotlib.cm.inferno
cmap = matplotlib.cm.viridis

cmap_norm = MidpointNormalize(midpoint=0.05, vmin=0, vmax=1)
vminmax = np.array([0, 1])


G_start=1
fs=6.5
#bar_ticks = [1,15,29,43]
#bar_ticks = [1,16,31,46,61]
bar_ticks = np.array([float(f'{x:.1f}') for x in 0.2*(np.array(range(6)))])



for one_tail in one_tails:
    if one_tail=='s':
        pdict['H']['mode'][0] = 'ci_s'
        pdict['H']['bound'][0] = ['both', 'both']
    elif one_tail=='d':
        pdict['H']['mode'][0] = 'bi_d'
        pdict['H']['bound'][0] = ['left', 'right']
    for tt_fix_a_lambda in tt_fix_a_lambdas:
        date_df = B20rA['vares'][f'tt_hb']['forecast']['vrs'][date_head].drop_duplicates()
        date_dict = [{date: [date_df.iloc[p][date]] for date in date_head} for p in range(date_df.shape[0])]

        a_u_s = np.sort(np.unique(B20rA['vares'][f'tt_hb']['forecast']['vrs']['a_u_1'].values))
        pdict['H']['L'] = a_u_s.size

        a_u_s_Ge = np.sort(np.unique(B20rA['vares'][f'cv_evt']['forecast']['vrs']['a_u'].values))
        pdict['Ge']['L'] = a_u_s_Ge.size

        
        a_q_s = np.sort(np.unique(B20rA['vares'][f'tt_hb']['forecast']['vrs']['a_q'].values))

        for mode in modes:

            pdict['H']['mode'][1] = mode

        
            for hb_dist in hb_dists:

                pdict['Ge']['dist'][0] = hb_dist

                pdict['H']['label'] = [f"{hwks_fns.get_part_symb(val='Hwk', tex=None)}_{{{pdict['H']['model'][h]}}}^{{{dist_symbol[hb_dist]}}}{{\\left(a_u\\right)}}" for h in range(pdict['H']['N'])]
                

                for o in o_evt:
                    if o==0:
                        pdict['G'] = copy.deepcopy(base_G)
                        if pdict['Ge']['dist'][0]=='norm':
                            G_start=1
                        else:
                            G_start=0

                    elif o==1:
                        pdict['G'] = copy.deepcopy(flip_G)
                        if pdict['G']['dist'][0]=='norm':
                            G_start=0
                        else:
                            G_start=1



                    if False:
                        pdict['G']['index'] = np.array([g*pdict['G']['L'] + np.min([G_start+g,pdict['Ge']['N']])*pdict['Ge']['L'] + np.max([G_start+g-pdict['Ge']['N'],0])*pdict['H']['L'] + np.array(range(pdict['G']['L'])) for g in range(pdict['G']['N'])])
                        pdict['Ge']['index'] = np.array([(1-G_start+ge)*pdict['G']['L'] + np.array(range(pdict['Ge']['L'])) for ge in range(pdict['Ge']['N'])])
                        pdict['H']['index'] = np.array([(1-G_start+pdict['Ge']['N']+h)*pdict['G']['L'] + pdict['Ge']['N']*pdict['Ge']['L'] + h*pdict['H']['L'] +  np.array(range(pdict['H']['L'])) for h in range(pdict['H']['N'])])
                    elif False:
                        pdict['G']['index'] = np.array([g*pdict['G']['L'] + np.array(range(pdict['G']['L'])) for g in range(pdict['G']['N'])])
                        pdict['Ge']['index'] = np.array([pdict['G']['N']*pdict['G']['L'] + ge*pdict['Ge']['L'] + np.array(range(pdict['Ge']['L'])) for ge in range(pdict['Ge']['N'])])
                        pdict['H']['index'] = np.array([pdict['G']['N']*pdict['G']['L'] + pdict['Ge']['N']*pdict['Ge']['L'] + h*pdict['H']['L'] + np.array(range(pdict['H']['L'])) for h in range(pdict['H']['N'])])
                    else:
                        pdict['H']['index'] = np.array([h*pdict['H']['L'] + np.array(range(pdict['H']['L'])) for h in range(pdict['H']['N'])])
                        pdict['Ge']['index'] = np.array([pdict['H']['N']*pdict['H']['L'] + ge*pdict['Ge']['L'] + np.array(range(pdict['Ge']['L'])) for ge in range(pdict['Ge']['N'])])
                        pdict['G']['index'] = np.array([pdict['H']['N']*pdict['H']['L'] + pdict['Ge']['N']*pdict['Ge']['L'] + g*pdict['G']['L'] + np.array(range(pdict['G']['L'])) for g in range(pdict['G']['N'])])



                    col_index = np.concatenate([
                            pdict['G']['index'][:,0],
                            np.concatenate(pdict['Ge']['index']),
                            np.concatenate(pdict['H']['index']),
                        ])
                    

                    pdict['Ge']['o'][0] = o
                    pdict['Ge']['label'] = [cvol_fns.calc_cv_symb(vol='GARCH', o=pdict['Ge']['o'][h], dist=pdict['Ge']['dist'][h], show_a_u_symb=True) for h in range(pdict['Ge']['N'])]

                    for p in range(len(date_dict)):
                        tick_std = calc_tick_std(ticks=ticks, train_end=date_dict[p]['train_end'][0], all_end=date_dict[p]['all_end'][0])

                        folder = f"ALL_{date_dict[p]['train_start'][0]}_{date_dict[p]['train_end'][0]}_{date_dict[p]['all_end'][0]}_tt_hb_cve{o}_fal{int(tt_fix_a_lambda)}_{mode}_{hb_dist}_{one_tail}"
                        #folder = f"ALL_Ge2T1T_test"
                        for period in periods:

                            for t in range(len(tests)):

                                test_name = tests[t]
                                y_val = f'CT_Q_{test_name}_p'

                                tt_test = np.full((len(ticks), len(tails), pdict['G']['N']*pdict['G']['L']+pdict['Ge']['N']*pdict['Ge']['L']+pdict['H']['N']*pdict['H']['L'], a_q_s.size), np.nan)
                                for i in range(len(ticks)):
                                    for j in range(len(tails)):
                                            
                                        if True:
                                            for g in range(pdict['G']['N']):
                                                tt_test[i,j,pdict['G']['index'][g,0],:] = np.array([misc_fns.index_df(B20rA['vares']['cv']['forecast']['vrs'], colval={
                                                                                                                'tick': [ticks[i]],
                                                                                                                **date_dict[p],
                                                                                                                'tail': [tails[j]],
                                                                                                                'vol': [pdict['G']['vol'][g]],
                                                                                                                'o': [pdict['G']['o'][g]],
                                                                                                                'dist': [pdict['G']['dist'][g]],
                                                                                                                'a_q': a_q_s,
                                                                                                                'period': [period],
                                                                                                            }).sort_values(by=['a_q'])[y_val].values])
            

                                            for ge in range(pdict['Ge']['N']):
                                                tt_test[i,j,pdict['Ge']['index'][ge,:],:] = np.array([misc_fns.index_df(B20rA['vares'][f'cv_evt']['forecast']['vrs'], colval={
                                                                                                                'tick': [ticks[i]],
                                                                                                                **date_dict[p],
                                                                                                                'tail': [tails[j]],
                                                                                                                'a_u': [a_u_s_Ge[pdict['Ge']['L']-1-k]],
                                                                                                                'vol': [pdict['Ge']['vol'][ge]],
                                                                                                                'o': [pdict['Ge']['o'][ge]],
                                                                                                                'dist': [pdict['Ge']['dist'][ge]],
                                                                                                                'a_q': a_q_s,
                                                                                                                'period': [period],
                                                                                                            }).sort_values(by=['a_q'])[y_val].values for k in range(pdict['Ge']['L'])])

                                            for h in range(pdict['H']['N']):
                                                tt_test[i,j,pdict['H']['index'][h,:],:] = np.array([misc_fns.index_df(B20rA['vares'][f'tt_hb']['forecast']['vrs'], colval={
                                                                                                                'tick': [ticks[i]],
                                                                                                                **date_dict[p],
                                                                                                                'tail': [tails[j]],
                                                                                                                'a_u_1': [a_u_s[pdict['H']['L']-1-k]],
                                                                                                                'fix_a_lambda': [tt_fix_a_lambda],
                                                                                                                'mode': [pdict['H']['mode'][h]],
                                                                                                                'mark': [pdict['H']['mark'][h]],
                                                                                                                'dist': [hb_dist],
                                                                                                                'bound': [pdict['H']['bound'][h][j]],
                                                                                                                'a_q': a_q_s,
                                                                                                                'period': [period],
                                                                                                            }).sort_values(by=['a_q'])[y_val].values for k in range(pdict['H']['L'])])


                                            


                                        else:
                                            for a in range(tt_test.shape[3]):
                                                #tt_test[i,j,col_index,a] = np.array(range(col_index.size))
                                                tt_test[i,j,col_index,a] = scipy.stats.expon().rvs(col_index.size) * (tick_std[ticks[i]] * ((a_q_s[a]*(1 - a_q_s[a]))**(1/np.sqrt(2))))
                                                                        
                                                                    

                                        for g in range(pdict['G']['N']):
                                            tt_test[i,j,pdict['G']['index'][g,:],:] = tt_test[i,j,pdict['G']['index'][g,0],:]
                                            #tt_rank[i,j,pdict['G']['index'][g,:],:] = tt_rank[i,j,pdict['G']['index'][g,0],:]
            


                                if make_plot:

                                    rep_i = len(ticks)//max_i

                                    for r in range(1):
                                        for s in range(rep_i):
                                            base_i = max_i*s

                                            fig_test = im6_plot_setup(name=folder+f"_{y_val[:-2]}_{period}_p_T{1+s}", nrows=max_i, ncols=len(tails), pb_ratio=6)

                                            dy = 1
                                            dx = (a_q_s[1]-a_q_s[0])/2
                                            

                                            extent = np.array([  
                                                        [a_q_s[0]-dx, a_q_s[-1]+dx], 
                                                        [0, 2*dy*(pdict['G']['N']*pdict['G']['L'] + pdict['Ge']['N']*pdict['Ge']['L'] + pdict['H']['N']*pdict['H']['L'])]
                                                    ])

                                            dH = (a_u_s[1] - a_u_s[0])/2
                                            H_range = np.array([a_u_s[0]-dH, a_u_s[-1]+dH])

                                            dH_Ge = (a_u_s_Ge[1] - a_u_s_Ge[0])/2
                                            H_Ge_range = np.array([a_u_s_Ge[0]-dH_Ge, a_u_s_Ge[-1]+dH_Ge])

                                            if False:
                                                g_ticks = [(2*tt_test.shape[2]-(np.sum(pdict['G']['index'][g,:][[0,-1]])+1)) for g in range(pdict['G']['N'])]
                                                g_labels = [f"${l}$" for l in pdict['G']['label']]
                                            else:
                                                g_ticks = []
                                                g_labels = []
                                            ge_ticks = 2*np.concatenate([(tt_test.shape[2]-(pdict['Ge']['index'][ge,-1]+1)) + pdict['Ge']['L']*(np.array(a_lambda_tick_Ge)-H_Ge_range[0])/np.diff(H_Ge_range)[0] for ge in range(pdict['Ge']['N'])])
                                            h_ticks = 2*np.concatenate([(tt_test.shape[2]-(pdict['H']['index'][h,-1]+1)) + pdict['H']['L']*(np.array(a_lambda_tick)-H_range[0])/np.diff(H_range)[0] for h in range(pdict['H']['N'])])

                                            ge_labels = np.concatenate([a_lambda_tick_Ge for h in range(pdict['Ge']['N'])])
                                            h_labels = np.concatenate([a_lambda_tick for h in range(pdict['H']['N'])])

                                            gh_ticks = np.concatenate([g_ticks, ge_ticks, h_ticks])
                                            gh_labels = np.concatenate([g_labels, ge_labels, h_labels])
                                            

                                            for i in range(max_i):
                                                if r==0:
                                                    i_s = base_i + i
                                                else:
                                                    i_s = s + 2*i
                                                for j in range(len(tt_test[i_s])):

                                                    tt_plot = tt_test[i_s,j,:,:]
                                                    
                                                    extent_key = [0,1,2,3] 
                                                    if flip_right:
                                                        if j==1:
                                                            tt_plot = np.flip(tt_plot, axis=1)
                                                            extent_key = [1,0,2,3] 

                                                    fig_test['ax'][i][j].imshow(tt_plot, aspect=2.35*0.95*(8.5/5.5)*(8.5/7)*(1/3)*(extent[0,1]-extent[0,0])/(extent[1,1]-extent[1,0]), 
                                                                                    #aspect=2.45*0.95*(8.5/5.5)*(8.5/7)*(1/3)*(extent[0,1]-extent[0,0])/(extent[1,1]-extent[1,0]), 
                                                                                    extent=np.concatenate(extent)[extent_key],
                                                                                    cmap=cmap, norm=cmap_norm,
                                                                                )
                                                    for ax in ['x', 'y']:
                                                        fig_test['ax'][i][j].tick_params(axis=ax, labelsize=fs)



                                                    for mod in pdict:
                                                        for k in range(pdict[mod]['index'].shape[0]):
                                                            fig_test['ax'][i][j].axhline(2*(tt_test.shape[2]-pdict[mod]['index'][k,0]), color='black', linewidth=1)
                                                            fig_test['ax'][i][j].axhline(2*(tt_test.shape[2]-(pdict[mod]['index'][k,-1]+1)), color='black', linewidth=1)

                                                    fig_test['ax'][i][j].set_yticks(gh_ticks)      
                                                    fig_test['ax'][i][j].set_yticklabels(gh_labels, fontsize=fs) 

                                                                
                                                fig_test['ax'][i][0].annotate(f"$\\mathbf{{({chr(ord('`')+1+i)})}}$ {ticks[i_s]}",
                                                                                                xy=(-0.35,  0.95), 
                                                                                                xycoords='axes fraction',
                                                                                                horizontalalignment='left',
                                                                                                verticalalignment='bottom',
                                                                                                rotation=0,
                                                                                                fontsize=8.5) 

                                                if False:
                                                    fig_test['ax'][i][0].annotate(ticks[i_s],
                                                                                            xy=(1.03, 0.5), xycoords='axes fraction',
                                                                                            horizontalalignment='left',
                                                                                            verticalalignment='center',
                                                                                            rotation=90,
                                                                                            fontsize=8.5)

                                                for mod in pdict:
                                                    y_tick = np.array([(2*tt_test.shape[2]-(np.sum(pdict[mod]['index'][h,:][[0,-1]])+1)) for h in range(pdict[mod]['N'])]) / (2*tt_test.shape[2])

                                                    for h in range(pdict[mod]['N']):
                                                        if np.isin(mod, ['Ge', 'H']):
                                                            fig_test['ax'][i][0].annotate(f"$a_{{u}}$",
                                                                                                xy=(-0.14, y_tick[h]), xycoords='axes fraction',
                                                                                                horizontalalignment='right',
                                                                                                verticalalignment='center',
                                                                                                rotation=90,
                                                                                                fontsize=fs)
                                                            fig_test['ax'][i][0].annotate(#f"$H{{\\left({pdict['H']['model'][-(1+h)]},{dist_symbol[hb_dist]}\\right)}}$",
                                                                                            f"${pdict[mod]['label'][h]}$",
                                                                                                xy=(-0.2,  y_tick[h]), xycoords='axes fraction',
                                                                                                horizontalalignment='right',
                                                                                                verticalalignment='center',
                                                                                                rotation=0,
                                                                                                fontsize=fs) 
                                                        else:
                                                            fig_test['ax'][i][0].annotate(#f"$H{{\\left({pdict['H']['model'][-(1+h)]},{dist_symbol[hb_dist]}\\right)}}$",
                                                                                        f"${pdict[mod]['label'][h]}$",
                                                                                            xy=(-0.2,  y_tick[h]), xycoords='axes fraction',
                                                                                            horizontalalignment='right',
                                                                                            verticalalignment='center',
                                                                                            rotation=0,
                                                                                            fontsize=fs) 

                    


                                                fig_test['ax'][i][-1].tick_params(labelright=False)


                                            for j in range(len(tails)):
                                                fig_test['ax'][0][j].set_xlabel(f'{tails[j][0].upper()}{tails[j][1:]}-tail $\\{tails[j]}arrow}}$', fontsize=8.5)
                                                fig_test['ax'][-1][j].set_xlabel(f'$a_{{q}}$', fontsize=fs)
                                                fig_test['ax'][0][j].tick_params(labeltop=False)


                                            for i in range(max_i):
                                                for j in range(len(tails)):
                                                    fig_test['ax'][i][j].set_xticks(a_u_ticks)
                                                    fig_test['ax'][i][j].tick_params(left=True, right=True, top=True, bottom=True,
                                                                                    labelleft=False, labelright=False, labeltop=False, labelbottom=False)
                                                fig_test['ax'][i][0].tick_params(labelleft=True)
                                            for j in range(len(tails)):
                                                fig_test['ax'][-1][j].tick_params(labelbottom=True)

                                        # fig_test['bar'].imshow(np.transpose([np.flip(np.linspace(0,1,1001))]), extent=[0,1,0,1], aspect=(8.5/7)*30,
                                        #                     cmap=cmap, norm=cmap_norm)
                                        # fig_test['bar'].set_ylabel(f'$p_{{{test_name}}}$', labelpad=-6)
                                            fig_test['bar'].imshow(np.transpose([np.flip(np.linspace(0,1,1001))]), extent=[0,1,0,1], aspect=1.2*(8.5/7)*30,
                                                        cmap=cmap, norm=cmap_norm)
                                            fig_test['bar'].set_ylabel(f'$p_{{\mathrm{{{test_name}}}}}$', labelpad=-6, fontsize=fs)
                                            fig_test['bar'].set_yticks(bar_ticks)
                                            fig_test['bar'].set_yticklabels(bar_ticks, fontsize=fs)


                                                
                                            fig_test['fig'].set_size_inches(5.7,6.5)
                                            fig_test['fig'].tight_layout()
                                            

                                            fstmp.fig_export(fig_test, width=5.7, height=6.5, 
                                                       #postsubs=folder
                                                       )

                        
uc_test = copy.deepcopy(tt_test)

In [ ]:
tests = ['CC']
#tests = ['UC']


#cmap = matplotlib.cm.inferno
cmap = matplotlib.cm.viridis

cmap_norm = MidpointNormalize(midpoint=0.05, vmin=0, vmax=1)
vminmax = np.array([0, 1])


G_start=1
fs=6.5
#bar_ticks = [1,15,29,43]
#bar_ticks = [1,16,31,46,61]
bar_ticks = np.array([float(f'{x:.1f}') for x in 0.2*(np.array(range(6)))])



for one_tail in one_tails:
    if one_tail=='s':
        pdict['H']['mode'][0] = 'ci_s'
        pdict['H']['bound'][0] = ['both', 'both']
    elif one_tail=='d':
        pdict['H']['mode'][0] = 'bi_d'
        pdict['H']['bound'][0] = ['left', 'right']
    for tt_fix_a_lambda in tt_fix_a_lambdas:
        date_df = B20rA['vares'][f'tt_hb']['forecast']['vrs'][date_head].drop_duplicates()
        date_dict = [{date: [date_df.iloc[p][date]] for date in date_head} for p in range(date_df.shape[0])]

        a_u_s = np.sort(np.unique(B20rA['vares'][f'tt_hb']['forecast']['vrs']['a_u_1'].values))
        pdict['H']['L'] = a_u_s.size

        a_u_s_Ge = np.sort(np.unique(B20rA['vares'][f'cv_evt']['forecast']['vrs']['a_u'].values))
        pdict['Ge']['L'] = a_u_s_Ge.size

        
        a_q_s = np.sort(np.unique(B20rA['vares'][f'tt_hb']['forecast']['vrs']['a_q'].values))

        for mode in modes:

            pdict['H']['mode'][1] = mode

        
            for hb_dist in hb_dists:

                pdict['Ge']['dist'][0] = hb_dist

                pdict['H']['label'] = [f"{hwks_fns.get_part_symb(val='Hwk', tex=None)}_{{{pdict['H']['model'][h]}}}^{{{dist_symbol[hb_dist]}}}{{\\left(a_u\\right)}}" for h in range(pdict['H']['N'])]
                

                for o in o_evt:
                    if o==0:
                        pdict['G'] = copy.deepcopy(base_G)
                        if pdict['Ge']['dist'][0]=='norm':
                            G_start=1
                        else:
                            G_start=0

                    elif o==1:
                        pdict['G'] = copy.deepcopy(flip_G)
                        if pdict['G']['dist'][0]=='norm':
                            G_start=0
                        else:
                            G_start=1



                    if False:
                        pdict['G']['index'] = np.array([g*pdict['G']['L'] + np.min([G_start+g,pdict['Ge']['N']])*pdict['Ge']['L'] + np.max([G_start+g-pdict['Ge']['N'],0])*pdict['H']['L'] + np.array(range(pdict['G']['L'])) for g in range(pdict['G']['N'])])
                        pdict['Ge']['index'] = np.array([(1-G_start+ge)*pdict['G']['L'] + np.array(range(pdict['Ge']['L'])) for ge in range(pdict['Ge']['N'])])
                        pdict['H']['index'] = np.array([(1-G_start+pdict['Ge']['N']+h)*pdict['G']['L'] + pdict['Ge']['N']*pdict['Ge']['L'] + h*pdict['H']['L'] +  np.array(range(pdict['H']['L'])) for h in range(pdict['H']['N'])])
                    elif False:
                        pdict['G']['index'] = np.array([g*pdict['G']['L'] + np.array(range(pdict['G']['L'])) for g in range(pdict['G']['N'])])
                        pdict['Ge']['index'] = np.array([pdict['G']['N']*pdict['G']['L'] + ge*pdict['Ge']['L'] + np.array(range(pdict['Ge']['L'])) for ge in range(pdict['Ge']['N'])])
                        pdict['H']['index'] = np.array([pdict['G']['N']*pdict['G']['L'] + pdict['Ge']['N']*pdict['Ge']['L'] + h*pdict['H']['L'] + np.array(range(pdict['H']['L'])) for h in range(pdict['H']['N'])])
                    else:
                        pdict['H']['index'] = np.array([h*pdict['H']['L'] + np.array(range(pdict['H']['L'])) for h in range(pdict['H']['N'])])
                        pdict['Ge']['index'] = np.array([pdict['H']['N']*pdict['H']['L'] + ge*pdict['Ge']['L'] + np.array(range(pdict['Ge']['L'])) for ge in range(pdict['Ge']['N'])])
                        pdict['G']['index'] = np.array([pdict['H']['N']*pdict['H']['L'] + pdict['Ge']['N']*pdict['Ge']['L'] + g*pdict['G']['L'] + np.array(range(pdict['G']['L'])) for g in range(pdict['G']['N'])])



                    col_index = np.concatenate([
                            pdict['G']['index'][:,0],
                            np.concatenate(pdict['Ge']['index']),
                            np.concatenate(pdict['H']['index']),
                        ])
                    

                    pdict['Ge']['o'][0] = o
                    pdict['Ge']['label'] = [cvol_fns.calc_cv_symb(vol='GARCH', o=pdict['Ge']['o'][h], dist=pdict['Ge']['dist'][h], show_a_u_symb=True) for h in range(pdict['Ge']['N'])]

                    for p in range(len(date_dict)):
                        tick_std = calc_tick_std(ticks=ticks, train_end=date_dict[p]['train_end'][0], all_end=date_dict[p]['all_end'][0])

                        folder = f"ALL_{date_dict[p]['train_start'][0]}_{date_dict[p]['train_end'][0]}_{date_dict[p]['all_end'][0]}_tt_hb_cve{o}_fal{int(tt_fix_a_lambda)}_{mode}_{hb_dist}_{one_tail}"
                        #folder = f"ALL_Ge2T1T_test"
                        for period in periods:

                            for t in range(len(tests)):

                                test_name = tests[t]
                                y_val = f'CT_Q_{test_name}_p'

                                tt_test = np.full((len(ticks), len(tails), pdict['G']['N']*pdict['G']['L']+pdict['Ge']['N']*pdict['Ge']['L']+pdict['H']['N']*pdict['H']['L'], a_q_s.size), np.nan)
                                for i in range(len(ticks)):
                                    for j in range(len(tails)):
                                            
                                        if True:
                                            for g in range(pdict['G']['N']):
                                                tt_test[i,j,pdict['G']['index'][g,0],:] = np.array([misc_fns.index_df(B20rA['vares']['cv']['forecast']['vrs'], colval={
                                                                                                                'tick': [ticks[i]],
                                                                                                                **date_dict[p],
                                                                                                                'tail': [tails[j]],
                                                                                                                'vol': [pdict['G']['vol'][g]],
                                                                                                                'o': [pdict['G']['o'][g]],
                                                                                                                'dist': [pdict['G']['dist'][g]],
                                                                                                                'a_q': a_q_s,
                                                                                                                'period': [period],
                                                                                                            }).sort_values(by=['a_q'])[y_val].values])
            

                                            for ge in range(pdict['Ge']['N']):
                                                tt_test[i,j,pdict['Ge']['index'][ge,:],:] = np.array([misc_fns.index_df(B20rA['vares'][f'cv_evt']['forecast']['vrs'], colval={
                                                                                                                'tick': [ticks[i]],
                                                                                                                **date_dict[p],
                                                                                                                'tail': [tails[j]],
                                                                                                                'a_u': [a_u_s_Ge[pdict['Ge']['L']-1-k]],
                                                                                                                'vol': [pdict['Ge']['vol'][ge]],
                                                                                                                'o': [pdict['Ge']['o'][ge]],
                                                                                                                'dist': [pdict['Ge']['dist'][ge]],
                                                                                                                'a_q': a_q_s,
                                                                                                                'period': [period],
                                                                                                            }).sort_values(by=['a_q'])[y_val].values for k in range(pdict['Ge']['L'])])

                                            for h in range(pdict['H']['N']):
                                                tt_test[i,j,pdict['H']['index'][h,:],:] = np.array([misc_fns.index_df(B20rA['vares'][f'tt_hb']['forecast']['vrs'], colval={
                                                                                                                'tick': [ticks[i]],
                                                                                                                **date_dict[p],
                                                                                                                'tail': [tails[j]],
                                                                                                                'a_u_1': [a_u_s[pdict['H']['L']-1-k]],
                                                                                                                'fix_a_lambda': [tt_fix_a_lambda],
                                                                                                                'mode': [pdict['H']['mode'][h]],
                                                                                                                'mark': [pdict['H']['mark'][h]],
                                                                                                                'dist': [hb_dist],
                                                                                                                'bound': [pdict['H']['bound'][h][j]],
                                                                                                                'a_q': a_q_s,
                                                                                                                'period': [period],
                                                                                                            }).sort_values(by=['a_q'])[y_val].values for k in range(pdict['H']['L'])])


                                            


                                        else:
                                            for a in range(tt_test.shape[3]):
                                                #tt_test[i,j,col_index,a] = np.array(range(col_index.size))
                                                tt_test[i,j,col_index,a] = scipy.stats.expon().rvs(col_index.size) * (tick_std[ticks[i]] * ((a_q_s[a]*(1 - a_q_s[a]))**(1/np.sqrt(2))))
                                                                        
                                                                    

                                        for g in range(pdict['G']['N']):
                                            tt_test[i,j,pdict['G']['index'][g,:],:] = tt_test[i,j,pdict['G']['index'][g,0],:]
                                            #tt_rank[i,j,pdict['G']['index'][g,:],:] = tt_rank[i,j,pdict['G']['index'][g,0],:]
            


                                if make_plot:

                                    rep_i = len(ticks)//max_i

                                    for r in range(1):
                                        for s in range(rep_i):
                                            base_i = max_i*s

                                            fig_test = im6_plot_setup(name=folder+f"_{y_val[:-2]}_{period}_p_T{1+s}", nrows=max_i, ncols=len(tails), pb_ratio=6)

                                            dy = 1
                                            dx = (a_q_s[1]-a_q_s[0])/2
                                            

                                            extent = np.array([  
                                                        [a_q_s[0]-dx, a_q_s[-1]+dx], 
                                                        [0, 2*dy*(pdict['G']['N']*pdict['G']['L'] + pdict['Ge']['N']*pdict['Ge']['L'] + pdict['H']['N']*pdict['H']['L'])]
                                                    ])

                                            dH = (a_u_s[1] - a_u_s[0])/2
                                            H_range = np.array([a_u_s[0]-dH, a_u_s[-1]+dH])

                                            dH_Ge = (a_u_s_Ge[1] - a_u_s_Ge[0])/2
                                            H_Ge_range = np.array([a_u_s_Ge[0]-dH_Ge, a_u_s_Ge[-1]+dH_Ge])

                                            if False:
                                                g_ticks = [(2*tt_test.shape[2]-(np.sum(pdict['G']['index'][g,:][[0,-1]])+1)) for g in range(pdict['G']['N'])]
                                                g_labels = [f"${l}$" for l in pdict['G']['label']]
                                            else:
                                                g_ticks = []
                                                g_labels = []
                                            ge_ticks = 2*np.concatenate([(tt_test.shape[2]-(pdict['Ge']['index'][ge,-1]+1)) + pdict['Ge']['L']*(np.array(a_lambda_tick_Ge)-H_Ge_range[0])/np.diff(H_Ge_range)[0] for ge in range(pdict['Ge']['N'])])
                                            h_ticks = 2*np.concatenate([(tt_test.shape[2]-(pdict['H']['index'][h,-1]+1)) + pdict['H']['L']*(np.array(a_lambda_tick)-H_range[0])/np.diff(H_range)[0] for h in range(pdict['H']['N'])])

                                            ge_labels = np.concatenate([a_lambda_tick_Ge for h in range(pdict['Ge']['N'])])
                                            h_labels = np.concatenate([a_lambda_tick for h in range(pdict['H']['N'])])

                                            gh_ticks = np.concatenate([g_ticks, ge_ticks, h_ticks])
                                            gh_labels = np.concatenate([g_labels, ge_labels, h_labels])
                                            

                                            for i in range(max_i):
                                                if r==0:
                                                    i_s = base_i + i
                                                else:
                                                    i_s = s + 2*i
                                                for j in range(len(tt_test[i_s])):

                                                    tt_plot = tt_test[i_s,j,:,:]
                                                    
                                                    extent_key = [0,1,2,3] 
                                                    if flip_right:
                                                        if j==1:
                                                            tt_plot = np.flip(tt_plot, axis=1)
                                                            extent_key = [1,0,2,3] 

                                                    fig_test['ax'][i][j].imshow(tt_plot, aspect=2.35*0.95*(8.5/5.5)*(8.5/7)*(1/3)*(extent[0,1]-extent[0,0])/(extent[1,1]-extent[1,0]), 
                                                                                    #aspect=2.45*0.95*(8.5/5.5)*(8.5/7)*(1/3)*(extent[0,1]-extent[0,0])/(extent[1,1]-extent[1,0]), 
                                                                                    extent=np.concatenate(extent)[extent_key],
                                                                                    cmap=cmap, norm=cmap_norm,
                                                                                )
                                                    for ax in ['x', 'y']:
                                                        fig_test['ax'][i][j].tick_params(axis=ax, labelsize=fs)



                                                    for mod in pdict:
                                                        for k in range(pdict[mod]['index'].shape[0]):
                                                            fig_test['ax'][i][j].axhline(2*(tt_test.shape[2]-pdict[mod]['index'][k,0]), color='black', linewidth=1)
                                                            fig_test['ax'][i][j].axhline(2*(tt_test.shape[2]-(pdict[mod]['index'][k,-1]+1)), color='black', linewidth=1)

                                                    fig_test['ax'][i][j].set_yticks(gh_ticks)      
                                                    fig_test['ax'][i][j].set_yticklabels(gh_labels, fontsize=fs) 

                                                                
                                                fig_test['ax'][i][0].annotate(f"$\\mathbf{{({chr(ord('`')+1+i)})}}$ {ticks[i_s]}",
                                                                                                xy=(-0.35,  0.95), 
                                                                                                xycoords='axes fraction',
                                                                                                horizontalalignment='left',
                                                                                                verticalalignment='bottom',
                                                                                                rotation=0,
                                                                                                fontsize=8.5) 

                                                if False:
                                                    fig_test['ax'][i][0].annotate(ticks[i_s],
                                                                                            xy=(1.03, 0.5), xycoords='axes fraction',
                                                                                            horizontalalignment='left',
                                                                                            verticalalignment='center',
                                                                                            rotation=90,
                                                                                            fontsize=8.5)

                                                for mod in pdict:
                                                    y_tick = np.array([(2*tt_test.shape[2]-(np.sum(pdict[mod]['index'][h,:][[0,-1]])+1)) for h in range(pdict[mod]['N'])]) / (2*tt_test.shape[2])

                                                    for h in range(pdict[mod]['N']):
                                                        if np.isin(mod, ['Ge', 'H']):
                                                            fig_test['ax'][i][0].annotate(f"$a_{{u}}$",
                                                                                                xy=(-0.14, y_tick[h]), xycoords='axes fraction',
                                                                                                horizontalalignment='right',
                                                                                                verticalalignment='center',
                                                                                                rotation=90,
                                                                                                fontsize=fs)
                                                            fig_test['ax'][i][0].annotate(#f"$H{{\\left({pdict['H']['model'][-(1+h)]},{dist_symbol[hb_dist]}\\right)}}$",
                                                                                            f"${pdict[mod]['label'][h]}$",
                                                                                                xy=(-0.2,  y_tick[h]), xycoords='axes fraction',
                                                                                                horizontalalignment='right',
                                                                                                verticalalignment='center',
                                                                                                rotation=0,
                                                                                                fontsize=fs) 
                                                        else:
                                                            fig_test['ax'][i][0].annotate(#f"$H{{\\left({pdict['H']['model'][-(1+h)]},{dist_symbol[hb_dist]}\\right)}}$",
                                                                                        f"${pdict[mod]['label'][h]}$",
                                                                                            xy=(-0.2,  y_tick[h]), xycoords='axes fraction',
                                                                                            horizontalalignment='right',
                                                                                            verticalalignment='center',
                                                                                            rotation=0,
                                                                                            fontsize=fs) 

                    


                                                fig_test['ax'][i][-1].tick_params(labelright=False)


                                            for j in range(len(tails)):
                                                fig_test['ax'][0][j].set_xlabel(f'{tails[j][0].upper()}{tails[j][1:]}-tail $\\{tails[j]}arrow}}$', fontsize=8.5)
                                                fig_test['ax'][-1][j].set_xlabel(f'$a_{{q}}$', fontsize=fs)
                                                fig_test['ax'][0][j].tick_params(labeltop=False)


                                            for i in range(max_i):
                                                for j in range(len(tails)):
                                                    fig_test['ax'][i][j].set_xticks(a_u_ticks)
                                                    fig_test['ax'][i][j].tick_params(left=True, right=True, top=True, bottom=True,
                                                                                    labelleft=False, labelright=False, labeltop=False, labelbottom=False)
                                                fig_test['ax'][i][0].tick_params(labelleft=True)
                                            for j in range(len(tails)):
                                                fig_test['ax'][-1][j].tick_params(labelbottom=True)

                                        # fig_test['bar'].imshow(np.transpose([np.flip(np.linspace(0,1,1001))]), extent=[0,1,0,1], aspect=(8.5/7)*30,
                                        #                     cmap=cmap, norm=cmap_norm)
                                        # fig_test['bar'].set_ylabel(f'$p_{{{test_name}}}$', labelpad=-6)
                                            fig_test['bar'].imshow(np.transpose([np.flip(np.linspace(0,1,1001))]), extent=[0,1,0,1], aspect=1.2*(8.5/7)*30,
                                                        cmap=cmap, norm=cmap_norm)
                                            fig_test['bar'].set_ylabel(f'$p_{{\mathrm{{{test_name}}}}}$', labelpad=-6, fontsize=fs)
                                            fig_test['bar'].set_yticks(bar_ticks)
                                            fig_test['bar'].set_yticklabels(bar_ticks, fontsize=fs)


                                                
                                            fig_test['fig'].set_size_inches(5.7,6.5)
                                            fig_test['fig'].tight_layout()
                                            

                                            fstmp.fig_export(fig_test, width=5.7, height=6.5, 
                                                       #postsubs=folder
                                                       )

                        
uc_test = copy.deepcopy(tt_test)

In [ ]:
tests = ['DQ_4']
#tests = ['UC']


#cmap = matplotlib.cm.inferno
cmap = matplotlib.cm.viridis

cmap_norm = MidpointNormalize(midpoint=0.05, vmin=0, vmax=1)
vminmax = np.array([0, 1])


G_start=1
fs=6.5
#bar_ticks = [1,15,29,43]
#bar_ticks = [1,16,31,46,61]
bar_ticks = np.array([float(f'{x:.1f}') for x in 0.2*(np.array(range(6)))])



for one_tail in one_tails:
    if one_tail=='s':
        pdict['H']['mode'][0] = 'ci_s'
        pdict['H']['bound'][0] = ['both', 'both']
    elif one_tail=='d':
        pdict['H']['mode'][0] = 'bi_d'
        pdict['H']['bound'][0] = ['left', 'right']
    for tt_fix_a_lambda in tt_fix_a_lambdas:
        date_df = B20rA['vares'][f'tt_hb']['forecast']['vrs'][date_head].drop_duplicates()
        date_dict = [{date: [date_df.iloc[p][date]] for date in date_head} for p in range(date_df.shape[0])]

        a_u_s = np.sort(np.unique(B20rA['vares'][f'tt_hb']['forecast']['vrs']['a_u_1'].values))
        pdict['H']['L'] = a_u_s.size

        a_u_s_Ge = np.sort(np.unique(B20rA['vares'][f'cv_evt']['forecast']['vrs']['a_u'].values))
        pdict['Ge']['L'] = a_u_s_Ge.size

        
        a_q_s = np.sort(np.unique(B20rA['vares'][f'tt_hb']['forecast']['vrs']['a_q'].values))

        for mode in modes:

            pdict['H']['mode'][1] = mode

        
            for hb_dist in hb_dists:

                pdict['Ge']['dist'][0] = hb_dist

                pdict['H']['label'] = [f"{hwks_fns.get_part_symb(val='Hwk', tex=None)}_{{{pdict['H']['model'][h]}}}^{{{dist_symbol[hb_dist]}}}{{\\left(a_u\\right)}}" for h in range(pdict['H']['N'])]
                

                for o in o_evt:
                    if o==0:
                        pdict['G'] = copy.deepcopy(base_G)
                        if pdict['Ge']['dist'][0]=='norm':
                            G_start=1
                        else:
                            G_start=0

                    elif o==1:
                        pdict['G'] = copy.deepcopy(flip_G)
                        if pdict['G']['dist'][0]=='norm':
                            G_start=0
                        else:
                            G_start=1



                    if False:
                        pdict['G']['index'] = np.array([g*pdict['G']['L'] + np.min([G_start+g,pdict['Ge']['N']])*pdict['Ge']['L'] + np.max([G_start+g-pdict['Ge']['N'],0])*pdict['H']['L'] + np.array(range(pdict['G']['L'])) for g in range(pdict['G']['N'])])
                        pdict['Ge']['index'] = np.array([(1-G_start+ge)*pdict['G']['L'] + np.array(range(pdict['Ge']['L'])) for ge in range(pdict['Ge']['N'])])
                        pdict['H']['index'] = np.array([(1-G_start+pdict['Ge']['N']+h)*pdict['G']['L'] + pdict['Ge']['N']*pdict['Ge']['L'] + h*pdict['H']['L'] +  np.array(range(pdict['H']['L'])) for h in range(pdict['H']['N'])])
                    elif False:
                        pdict['G']['index'] = np.array([g*pdict['G']['L'] + np.array(range(pdict['G']['L'])) for g in range(pdict['G']['N'])])
                        pdict['Ge']['index'] = np.array([pdict['G']['N']*pdict['G']['L'] + ge*pdict['Ge']['L'] + np.array(range(pdict['Ge']['L'])) for ge in range(pdict['Ge']['N'])])
                        pdict['H']['index'] = np.array([pdict['G']['N']*pdict['G']['L'] + pdict['Ge']['N']*pdict['Ge']['L'] + h*pdict['H']['L'] + np.array(range(pdict['H']['L'])) for h in range(pdict['H']['N'])])
                    else:
                        pdict['H']['index'] = np.array([h*pdict['H']['L'] + np.array(range(pdict['H']['L'])) for h in range(pdict['H']['N'])])
                        pdict['Ge']['index'] = np.array([pdict['H']['N']*pdict['H']['L'] + ge*pdict['Ge']['L'] + np.array(range(pdict['Ge']['L'])) for ge in range(pdict['Ge']['N'])])
                        pdict['G']['index'] = np.array([pdict['H']['N']*pdict['H']['L'] + pdict['Ge']['N']*pdict['Ge']['L'] + g*pdict['G']['L'] + np.array(range(pdict['G']['L'])) for g in range(pdict['G']['N'])])



                    col_index = np.concatenate([
                            pdict['G']['index'][:,0],
                            np.concatenate(pdict['Ge']['index']),
                            np.concatenate(pdict['H']['index']),
                        ])
                    

                    pdict['Ge']['o'][0] = o
                    pdict['Ge']['label'] = [cvol_fns.calc_cv_symb(vol='GARCH', o=pdict['Ge']['o'][h], dist=pdict['Ge']['dist'][h], show_a_u_symb=True) for h in range(pdict['Ge']['N'])]

                    for p in range(len(date_dict)):
                        tick_std = calc_tick_std(ticks=ticks, train_end=date_dict[p]['train_end'][0], all_end=date_dict[p]['all_end'][0])

                        folder = f"ALL_{date_dict[p]['train_start'][0]}_{date_dict[p]['train_end'][0]}_{date_dict[p]['all_end'][0]}_tt_hb_cve{o}_fal{int(tt_fix_a_lambda)}_{mode}_{hb_dist}_{one_tail}"
                        #folder = f"ALL_Ge2T1T_test"
                        for period in periods:

                            for t in range(len(tests)):

                                test_name = tests[t]
                                y_val = f'CT_Q_{test_name}_p'

                                tt_test = np.full((len(ticks), len(tails), pdict['G']['N']*pdict['G']['L']+pdict['Ge']['N']*pdict['Ge']['L']+pdict['H']['N']*pdict['H']['L'], a_q_s.size), np.nan)
                                for i in range(len(ticks)):
                                    for j in range(len(tails)):
                                            
                                        if True:
                                            for g in range(pdict['G']['N']):
                                                tt_test[i,j,pdict['G']['index'][g,0],:] = np.array([misc_fns.index_df(B20rA['vares']['cv']['forecast']['vrs'], colval={
                                                                                                                'tick': [ticks[i]],
                                                                                                                **date_dict[p],
                                                                                                                'tail': [tails[j]],
                                                                                                                'vol': [pdict['G']['vol'][g]],
                                                                                                                'o': [pdict['G']['o'][g]],
                                                                                                                'dist': [pdict['G']['dist'][g]],
                                                                                                                'a_q': a_q_s,
                                                                                                                'period': [period],
                                                                                                            }).sort_values(by=['a_q'])[y_val].values])
            

                                            for ge in range(pdict['Ge']['N']):
                                                tt_test[i,j,pdict['Ge']['index'][ge,:],:] = np.array([misc_fns.index_df(B20rA['vares'][f'cv_evt']['forecast']['vrs'], colval={
                                                                                                                'tick': [ticks[i]],
                                                                                                                **date_dict[p],
                                                                                                                'tail': [tails[j]],
                                                                                                                'a_u': [a_u_s_Ge[pdict['Ge']['L']-1-k]],
                                                                                                                'vol': [pdict['Ge']['vol'][ge]],
                                                                                                                'o': [pdict['Ge']['o'][ge]],
                                                                                                                'dist': [pdict['Ge']['dist'][ge]],
                                                                                                                'a_q': a_q_s,
                                                                                                                'period': [period],
                                                                                                            }).sort_values(by=['a_q'])[y_val].values for k in range(pdict['Ge']['L'])])

                                            for h in range(pdict['H']['N']):
                                                tt_test[i,j,pdict['H']['index'][h,:],:] = np.array([misc_fns.index_df(B20rA['vares'][f'tt_hb']['forecast']['vrs'], colval={
                                                                                                                'tick': [ticks[i]],
                                                                                                                **date_dict[p],
                                                                                                                'tail': [tails[j]],
                                                                                                                'a_u_1': [a_u_s[pdict['H']['L']-1-k]],
                                                                                                                'fix_a_lambda': [tt_fix_a_lambda],
                                                                                                                'mode': [pdict['H']['mode'][h]],
                                                                                                                'mark': [pdict['H']['mark'][h]],
                                                                                                                'dist': [hb_dist],
                                                                                                                'bound': [pdict['H']['bound'][h][j]],
                                                                                                                'a_q': a_q_s,
                                                                                                                'period': [period],
                                                                                                            }).sort_values(by=['a_q'])[y_val].values for k in range(pdict['H']['L'])])


                                            


                                        else:
                                            for a in range(tt_test.shape[3]):
                                                #tt_test[i,j,col_index,a] = np.array(range(col_index.size))
                                                tt_test[i,j,col_index,a] = scipy.stats.expon().rvs(col_index.size) * (tick_std[ticks[i]] * ((a_q_s[a]*(1 - a_q_s[a]))**(1/np.sqrt(2))))
                                                                        
                                                                    

                                        for g in range(pdict['G']['N']):
                                            tt_test[i,j,pdict['G']['index'][g,:],:] = tt_test[i,j,pdict['G']['index'][g,0],:]
                                            #tt_rank[i,j,pdict['G']['index'][g,:],:] = tt_rank[i,j,pdict['G']['index'][g,0],:]
            


                                if make_plot:

                                    rep_i = len(ticks)//max_i

                                    for r in range(1):
                                        for s in range(rep_i):
                                            base_i = max_i*s

                                            fig_test = im6_plot_setup(name=folder+f"_{y_val[:-2]}_{period}_p_T{1+s}", nrows=max_i, ncols=len(tails), pb_ratio=6)

                                            dy = 1
                                            dx = (a_q_s[1]-a_q_s[0])/2
                                            

                                            extent = np.array([  
                                                        [a_q_s[0]-dx, a_q_s[-1]+dx], 
                                                        [0, 2*dy*(pdict['G']['N']*pdict['G']['L'] + pdict['Ge']['N']*pdict['Ge']['L'] + pdict['H']['N']*pdict['H']['L'])]
                                                    ])

                                            dH = (a_u_s[1] - a_u_s[0])/2
                                            H_range = np.array([a_u_s[0]-dH, a_u_s[-1]+dH])

                                            dH_Ge = (a_u_s_Ge[1] - a_u_s_Ge[0])/2
                                            H_Ge_range = np.array([a_u_s_Ge[0]-dH_Ge, a_u_s_Ge[-1]+dH_Ge])

                                            if False:
                                                g_ticks = [(2*tt_test.shape[2]-(np.sum(pdict['G']['index'][g,:][[0,-1]])+1)) for g in range(pdict['G']['N'])]
                                                g_labels = [f"${l}$" for l in pdict['G']['label']]
                                            else:
                                                g_ticks = []
                                                g_labels = []
                                            ge_ticks = 2*np.concatenate([(tt_test.shape[2]-(pdict['Ge']['index'][ge,-1]+1)) + pdict['Ge']['L']*(np.array(a_lambda_tick_Ge)-H_Ge_range[0])/np.diff(H_Ge_range)[0] for ge in range(pdict['Ge']['N'])])
                                            h_ticks = 2*np.concatenate([(tt_test.shape[2]-(pdict['H']['index'][h,-1]+1)) + pdict['H']['L']*(np.array(a_lambda_tick)-H_range[0])/np.diff(H_range)[0] for h in range(pdict['H']['N'])])

                                            ge_labels = np.concatenate([a_lambda_tick_Ge for h in range(pdict['Ge']['N'])])
                                            h_labels = np.concatenate([a_lambda_tick for h in range(pdict['H']['N'])])

                                            gh_ticks = np.concatenate([g_ticks, ge_ticks, h_ticks])
                                            gh_labels = np.concatenate([g_labels, ge_labels, h_labels])
                                            

                                            for i in range(max_i):
                                                if r==0:
                                                    i_s = base_i + i
                                                else:
                                                    i_s = s + 2*i
                                                for j in range(len(tt_test[i_s])):

                                                    tt_plot = tt_test[i_s,j,:,:]
                                                    
                                                    extent_key = [0,1,2,3] 
                                                    if flip_right:
                                                        if j==1:
                                                            tt_plot = np.flip(tt_plot, axis=1)
                                                            extent_key = [1,0,2,3] 

                                                    fig_test['ax'][i][j].imshow(tt_plot, aspect=2.35*0.95*(8.5/5.5)*(8.5/7)*(1/3)*(extent[0,1]-extent[0,0])/(extent[1,1]-extent[1,0]), 
                                                                                    #aspect=2.45*0.95*(8.5/5.5)*(8.5/7)*(1/3)*(extent[0,1]-extent[0,0])/(extent[1,1]-extent[1,0]), 
                                                                                    extent=np.concatenate(extent)[extent_key],
                                                                                    cmap=cmap, norm=cmap_norm,
                                                                                )
                                                    for ax in ['x', 'y']:
                                                        fig_test['ax'][i][j].tick_params(axis=ax, labelsize=fs)



                                                    for mod in pdict:
                                                        for k in range(pdict[mod]['index'].shape[0]):
                                                            fig_test['ax'][i][j].axhline(2*(tt_test.shape[2]-pdict[mod]['index'][k,0]), color='black', linewidth=1)
                                                            fig_test['ax'][i][j].axhline(2*(tt_test.shape[2]-(pdict[mod]['index'][k,-1]+1)), color='black', linewidth=1)

                                                    fig_test['ax'][i][j].set_yticks(gh_ticks)      
                                                    fig_test['ax'][i][j].set_yticklabels(gh_labels, fontsize=fs) 

                                                                
                                                fig_test['ax'][i][0].annotate(f"$\\mathbf{{({chr(ord('`')+1+i)})}}$ {ticks[i_s]}",
                                                                                                xy=(-0.35,  0.95), 
                                                                                                xycoords='axes fraction',
                                                                                                horizontalalignment='left',
                                                                                                verticalalignment='bottom',
                                                                                                rotation=0,
                                                                                                fontsize=8.5) 

                                                if False:
                                                    fig_test['ax'][i][0].annotate(ticks[i_s],
                                                                                            xy=(1.03, 0.5), xycoords='axes fraction',
                                                                                            horizontalalignment='left',
                                                                                            verticalalignment='center',
                                                                                            rotation=90,
                                                                                            fontsize=8.5)

                                                for mod in pdict:
                                                    y_tick = np.array([(2*tt_test.shape[2]-(np.sum(pdict[mod]['index'][h,:][[0,-1]])+1)) for h in range(pdict[mod]['N'])]) / (2*tt_test.shape[2])

                                                    for h in range(pdict[mod]['N']):
                                                        if np.isin(mod, ['Ge', 'H']):
                                                            fig_test['ax'][i][0].annotate(f"$a_{{u}}$",
                                                                                                xy=(-0.14, y_tick[h]), xycoords='axes fraction',
                                                                                                horizontalalignment='right',
                                                                                                verticalalignment='center',
                                                                                                rotation=90,
                                                                                                fontsize=fs)
                                                            fig_test['ax'][i][0].annotate(#f"$H{{\\left({pdict['H']['model'][-(1+h)]},{dist_symbol[hb_dist]}\\right)}}$",
                                                                                            f"${pdict[mod]['label'][h]}$",
                                                                                                xy=(-0.2,  y_tick[h]), xycoords='axes fraction',
                                                                                                horizontalalignment='right',
                                                                                                verticalalignment='center',
                                                                                                rotation=0,
                                                                                                fontsize=fs) 
                                                        else:
                                                            fig_test['ax'][i][0].annotate(#f"$H{{\\left({pdict['H']['model'][-(1+h)]},{dist_symbol[hb_dist]}\\right)}}$",
                                                                                        f"${pdict[mod]['label'][h]}$",
                                                                                            xy=(-0.2,  y_tick[h]), xycoords='axes fraction',
                                                                                            horizontalalignment='right',
                                                                                            verticalalignment='center',
                                                                                            rotation=0,
                                                                                            fontsize=fs) 

                    


                                                fig_test['ax'][i][-1].tick_params(labelright=False)


                                            for j in range(len(tails)):
                                                fig_test['ax'][0][j].set_xlabel(f'{tails[j][0].upper()}{tails[j][1:]}-tail $\\{tails[j]}arrow}}$', fontsize=8.5)
                                                fig_test['ax'][-1][j].set_xlabel(f'$a_{{q}}$', fontsize=fs)
                                                fig_test['ax'][0][j].tick_params(labeltop=False)


                                            for i in range(max_i):
                                                for j in range(len(tails)):
                                                    fig_test['ax'][i][j].set_xticks(a_u_ticks)
                                                    fig_test['ax'][i][j].tick_params(left=True, right=True, top=True, bottom=True,
                                                                                    labelleft=False, labelright=False, labeltop=False, labelbottom=False)
                                                fig_test['ax'][i][0].tick_params(labelleft=True)
                                            for j in range(len(tails)):
                                                fig_test['ax'][-1][j].tick_params(labelbottom=True)

                                        # fig_test['bar'].imshow(np.transpose([np.flip(np.linspace(0,1,1001))]), extent=[0,1,0,1], aspect=(8.5/7)*30,
                                        #                     cmap=cmap, norm=cmap_norm)
                                        # fig_test['bar'].set_ylabel(f'$p_{{{test_name}}}$', labelpad=-6)
                                            fig_test['bar'].imshow(np.transpose([np.flip(np.linspace(0,1,1001))]), extent=[0,1,0,1], aspect=1.2*(8.5/7)*30,
                                                        cmap=cmap, norm=cmap_norm)
                                            fig_test['bar'].set_ylabel(f'$p_{{\mathrm{{{test_name}}}}}$', labelpad=-6, fontsize=fs)
                                            fig_test['bar'].set_yticks(bar_ticks)
                                            fig_test['bar'].set_yticklabels(bar_ticks, fontsize=fs)


                                                
                                            fig_test['fig'].set_size_inches(5.7,6.5)
                                            fig_test['fig'].tight_layout()
                                            

                                            fstmp.fig_export(fig_test, width=5.7, height=6.5, 
                                                       #postsubs=folder
                                                       )

                        
uc_test = copy.deepcopy(tt_test)

In [ ]:
tests = ['ZMD_1000_0:0']


#cmap = matplotlib.cm.inferno
cmap_1 = matplotlib.cm.viridis
cmap_2 = matplotlib.cm.RdBu

cmap_norm_1 = MidpointNormalize(midpoint=0.05, vmin=0, vmax=1)
cmap_norm_2 = MultipointNormalize(vpoints=[0.025, 0.975], points=[1/4,3/4], vmin=0, vmax=1)
vminmax = np.array([0, 1])


cmap=cmap_2
cmap_norm=cmap_norm_2


#bar_ticks = [1,15,29,43]
#bar_ticks = [1,16,31,46,61]
bar_ticks = np.array([float(f'{x:.1f}') for x in 0.2*(np.array(range(6)))])



for one_tail in one_tails:
    if one_tail=='s':
        pdict['H']['mode'][0] = 'ci_s'
        pdict['H']['bound'][0] = ['both', 'both']
    elif one_tail=='d':
        pdict['H']['mode'][0] = 'bi_d'
        pdict['H']['bound'][0] = ['left', 'right']
    for tt_fix_a_lambda in tt_fix_a_lambdas:
        date_df = B20rA['vares'][f'tt_hb']['forecast']['vrs'][date_head].drop_duplicates()
        date_dict = [{date: [date_df.iloc[p][date]] for date in date_head} for p in range(date_df.shape[0])]

        a_u_s = np.sort(np.unique(B20rA['vares'][f'tt_hb']['forecast']['vrs']['a_u_1'].values))
        pdict['H']['L'] = a_u_s.size

        a_u_s_Ge = np.sort(np.unique(B20rA['vares'][f'cv_evt']['forecast']['vrs']['a_u'].values))
        pdict['Ge']['L'] = a_u_s_Ge.size

        
        a_q_s = np.sort(np.unique(B20rA['vares'][f'tt_hb']['forecast']['vrs']['a_q'].values))

        for mode in modes:

            pdict['H']['mode'][1] = mode

        
            for hb_dist in hb_dists:

                pdict['Ge']['dist'][0] = hb_dist

                pdict['H']['label'] = [f"{hwks_fns.get_part_symb(val='Hwk', tex=None)}_{{{pdict['H']['model'][h]}}}^{{{dist_symbol[hb_dist]}}}{{\\left(a_u\\right)}}" for h in range(pdict['H']['N'])]
                

                for o in o_evt:
                    if o==0:
                        pdict['G'] = copy.deepcopy(base_G)
                        if pdict['Ge']['dist'][0]=='norm':
                            G_start=1
                        else:
                            G_start=0

                    elif o==1:
                        pdict['G'] = copy.deepcopy(flip_G)
                        if pdict['G']['dist'][0]=='norm':
                            G_start=0
                        else:
                            G_start=1



                    if False:
                        pdict['G']['index'] = np.array([g*pdict['G']['L'] + np.min([G_start+g,pdict['Ge']['N']])*pdict['Ge']['L'] + np.max([G_start+g-pdict['Ge']['N'],0])*pdict['H']['L'] + np.array(range(pdict['G']['L'])) for g in range(pdict['G']['N'])])
                        pdict['Ge']['index'] = np.array([(1-G_start+ge)*pdict['G']['L'] + np.array(range(pdict['Ge']['L'])) for ge in range(pdict['Ge']['N'])])
                        pdict['H']['index'] = np.array([(1-G_start+pdict['Ge']['N']+h)*pdict['G']['L'] + pdict['Ge']['N']*pdict['Ge']['L'] + h*pdict['H']['L'] +  np.array(range(pdict['H']['L'])) for h in range(pdict['H']['N'])])
                    elif False:
                        pdict['G']['index'] = np.array([g*pdict['G']['L'] + np.array(range(pdict['G']['L'])) for g in range(pdict['G']['N'])])
                        pdict['Ge']['index'] = np.array([pdict['G']['N']*pdict['G']['L'] + ge*pdict['Ge']['L'] + np.array(range(pdict['Ge']['L'])) for ge in range(pdict['Ge']['N'])])
                        pdict['H']['index'] = np.array([pdict['G']['N']*pdict['G']['L'] + pdict['Ge']['N']*pdict['Ge']['L'] + h*pdict['H']['L'] + np.array(range(pdict['H']['L'])) for h in range(pdict['H']['N'])])
                    else:
                        pdict['H']['index'] = np.array([h*pdict['H']['L'] + np.array(range(pdict['H']['L'])) for h in range(pdict['H']['N'])])
                        pdict['Ge']['index'] = np.array([pdict['H']['N']*pdict['H']['L'] + ge*pdict['Ge']['L'] + np.array(range(pdict['Ge']['L'])) for ge in range(pdict['Ge']['N'])])
                        pdict['G']['index'] = np.array([pdict['H']['N']*pdict['H']['L'] + pdict['Ge']['N']*pdict['Ge']['L'] + g*pdict['G']['L'] + np.array(range(pdict['G']['L'])) for g in range(pdict['G']['N'])])



                    col_index = np.concatenate([
                            pdict['G']['index'][:,0],
                            np.concatenate(pdict['Ge']['index']),
                            np.concatenate(pdict['H']['index']),
                        ])
                    

                    pdict['Ge']['o'][0] = o
                    pdict['Ge']['label'] = [cvol_fns.calc_cv_symb(vol='GARCH', o=pdict['Ge']['o'][h], dist=pdict['Ge']['dist'][h], show_a_u_symb=True) for h in range(pdict['Ge']['N'])]

                    for p in range(len(date_dict)):
                        tick_std = calc_tick_std(ticks=ticks, train_end=date_dict[p]['train_end'][0], all_end=date_dict[p]['all_end'][0])

                        folder = f"ALL_{date_dict[p]['train_start'][0]}_{date_dict[p]['train_end'][0]}_{date_dict[p]['all_end'][0]}_tt_hb_fal{int(tt_fix_a_lambda)}_cve{o}_{mode}_{hb_dist}_{one_tail}"
                        #folder = f"ALL_Ge2T1T_test"
                        for period in periods:

                            for t in range(len(tests)):

                                test_name = tests[t]
                                y_val = f'CT_E_{test_name}_F'
                                test_label = test_name[:3]

                                tt_test = np.full((len(ticks), len(tails), pdict['G']['N']*pdict['G']['L']+pdict['Ge']['N']*pdict['Ge']['L']+pdict['H']['N']*pdict['H']['L'], a_q_s.size), np.nan)
                                for i in range(len(ticks)):
                                    for j in range(len(tails)):
                                            
                                        if True:
                                            for g in range(pdict['G']['N']):
                                                tt_test[i,j,pdict['G']['index'][g,0],:] = np.array([misc_fns.index_df(B20rA['vares']['cv']['forecast']['vrs'], colval={
                                                                                                                'tick': [ticks[i]],
                                                                                                                **date_dict[p],
                                                                                                                'tail': [tails[j]],
                                                                                                                'vol': [pdict['G']['vol'][g]],
                                                                                                                'o': [pdict['G']['o'][g]],
                                                                                                                'dist': [pdict['G']['dist'][g]],
                                                                                                                'a_q': a_q_s,
                                                                                                                'period': [period],
                                                                                                            }).sort_values(by=['a_q'])[y_val].values])
            

                                            for ge in range(pdict['Ge']['N']):
                                                tt_test[i,j,pdict['Ge']['index'][ge,:],:] = np.array([misc_fns.index_df(B20rA['vares'][f'cv_evt']['forecast']['vrs'], colval={
                                                                                                                'tick': [ticks[i]],
                                                                                                                **date_dict[p],
                                                                                                                'tail': [tails[j]],
                                                                                                                'a_u': [a_u_s_Ge[pdict['Ge']['L']-1-k]],
                                                                                                                'vol': [pdict['Ge']['vol'][ge]],
                                                                                                                'o': [pdict['Ge']['o'][ge]],
                                                                                                                'dist': [pdict['Ge']['dist'][ge]],
                                                                                                                'a_q': a_q_s,
                                                                                                                'period': [period],
                                                                                                            }).sort_values(by=['a_q'])[y_val].values for k in range(pdict['Ge']['L'])])

                                            for h in range(pdict['H']['N']):
                                                tt_test[i,j,pdict['H']['index'][h,:],:] = np.array([misc_fns.index_df(B20rA['vares'][f'tt_hb']['forecast']['vrs'], colval={
                                                                                                                'tick': [ticks[i]],
                                                                                                                **date_dict[p],
                                                                                                                'tail': [tails[j]],
                                                                                                                'a_u_1': [a_u_s[pdict['H']['L']-1-k]],
                                                                                                                'fix_a_lambda': [tt_fix_a_lambda],
                                                                                                                'mode': [pdict['H']['mode'][h]],
                                                                                                                'mark': [pdict['H']['mark'][h]],
                                                                                                                'dist': [hb_dist],
                                                                                                                'bound': [pdict['H']['bound'][h][j]],
                                                                                                                'a_q': a_q_s,
                                                                                                                'period': [period],
                                                                                                            }).sort_values(by=['a_q'])[y_val].values for k in range(pdict['H']['L'])])


                                            


                                        else:
                                            for a in range(tt_test.shape[3]):
                                                #tt_test[i,j,col_index,a] = np.array(range(col_index.size))
                                                tt_test[i,j,col_index,a] = scipy.stats.expon().rvs(col_index.size) * (tick_std[ticks[i]] * ((a_q_s[a]*(1 - a_q_s[a]))**(1/np.sqrt(2))))
                                                                        
                                                                    

                                        for g in range(pdict['G']['N']):
                                            tt_test[i,j,pdict['G']['index'][g,:],:] = tt_test[i,j,pdict['G']['index'][g,0],:]
                                            #tt_rank[i,j,pdict['G']['index'][g,:],:] = tt_rank[i,j,pdict['G']['index'][g,0],:]
            


                                if make_plot:

                                    rep_i = len(ticks)//max_i

                                    for val in ['F', 'pLR', 'pL', 'pR']:
                                        if val=='F':
                                            cmap=cmap_2
                                            cmap_norm=cmap_norm_2
                                            bar_label = f'$F_{{{test_label}}}$'
                                        else:
                                            cmap=cmap_1
                                            cmap_norm=cmap_norm_1
                                            if val=='pLR':
                                                bar_label = f'$p_{{{test_label}}}^{{\\leftrightarrow}}$'
                                            if val=='pL':
                                                bar_label = f'$p_{{{test_label}}}^{{\\leftarrow}}$'
                                            if val=='pR':
                                                bar_label = f'$p_{{{test_label}}}^{{\\rightarrow}}$'

                                        

                                        for r in range(1):
                                            for s in range(rep_i):
                                                base_i = max_i*s

                                                fig_test = im6_plot_setup(name=folder+f"_CT_E_{test_label}_{period}_{val}_T{1+s}", nrows=max_i, ncols=len(tails), pb_ratio=6)

                                                dy = 1
                                                dx = (a_q_s[1]-a_q_s[0])/2
                                                

                                                extent = np.array([  
                                                            [a_q_s[0]-dx, a_q_s[-1]+dx], 
                                                            [0, 2*dy*(pdict['G']['N']*pdict['G']['L'] + pdict['Ge']['N']*pdict['Ge']['L'] + pdict['H']['N']*pdict['H']['L'])]
                                                        ])

                                                dH = (a_u_s[1] - a_u_s[0])/2
                                                H_range = np.array([a_u_s[0]-dH, a_u_s[-1]+dH])

                                                dH_Ge = (a_u_s_Ge[1] - a_u_s_Ge[0])/2
                                                H_Ge_range = np.array([a_u_s_Ge[0]-dH_Ge, a_u_s_Ge[-1]+dH_Ge])

                                                if False:
                                                    g_ticks = [(2*tt_test.shape[2]-(np.sum(pdict['G']['index'][g,:][[0,-1]])+1)) for g in range(pdict['G']['N'])]
                                                    g_labels = [f"${l}$" for l in pdict['G']['label']]
                                                else:
                                                    g_ticks = []
                                                    g_labels = []
                                                ge_ticks = 2*np.concatenate([(tt_test.shape[2]-(pdict['Ge']['index'][ge,-1]+1)) + pdict['Ge']['L']*(np.array(a_lambda_tick_Ge)-H_Ge_range[0])/np.diff(H_Ge_range)[0] for ge in range(pdict['Ge']['N'])])
                                                h_ticks = 2*np.concatenate([(tt_test.shape[2]-(pdict['H']['index'][h,-1]+1)) + pdict['H']['L']*(np.array(a_lambda_tick)-H_range[0])/np.diff(H_range)[0] for h in range(pdict['H']['N'])])

                                                ge_labels = np.concatenate([a_lambda_tick_Ge for h in range(pdict['Ge']['N'])])
                                                h_labels = np.concatenate([a_lambda_tick for h in range(pdict['H']['N'])])

                                                gh_ticks = np.concatenate([g_ticks, ge_ticks, h_ticks])
                                                gh_labels = np.concatenate([g_labels, ge_labels, h_labels])
                                                

                                                for i in range(max_i):
                                                    if r==0:
                                                        i_s = base_i + i
                                                    else:
                                                        i_s = s + 2*i
                                                    for j in range(len(tt_test[i_s])):

                                                        tt_plot = tt_test[i_s,j,:,:]

                                                        if val=='pLR':
                                                            tt_plot = 1 - 2*np.abs(tt_plot - 0.5)
                                                        elif val=='pR':
                                                            tt_plot = 1 - tt_plot
                                                        
                                                        extent_key = [0,1,2,3] 
                                                        if flip_right:
                                                            if j==1:
                                                                tt_plot = np.flip(tt_plot, axis=1)
                                                                extent_key = [1,0,2,3] 

                                                        fig_test['ax'][i][j].imshow(tt_plot, aspect=2.35*0.95*(8.5/5.5)*(8.5/7)*(1/3)*(extent[0,1]-extent[0,0])/(extent[1,1]-extent[1,0]), 
                                                                                    #aspect=2.45*0.95*(8.5/5.5)*(8.5/7)*(1/3)*(extent[0,1]-extent[0,0])/(extent[1,1]-extent[1,0]), 
                                                                                    extent=np.concatenate(extent)[extent_key],
                                                                                    cmap=cmap, norm=cmap_norm,
                                                                                )
                                                        for ax in ['x', 'y']:
                                                            fig_test['ax'][i][j].tick_params(axis=ax, labelsize=fs)



                                                        for mod in pdict:
                                                            for k in range(pdict[mod]['index'].shape[0]):
                                                                fig_test['ax'][i][j].axhline(2*(tt_test.shape[2]-pdict[mod]['index'][k,0]), color='black', linewidth=1)
                                                                fig_test['ax'][i][j].axhline(2*(tt_test.shape[2]-(pdict[mod]['index'][k,-1]+1)), color='black', linewidth=1)

                                                        fig_test['ax'][i][j].set_yticks(gh_ticks)      
                                                        fig_test['ax'][i][j].set_yticklabels(gh_labels, fontsize=fs) 

                                                    fig_test['ax'][i][0].annotate(f"$\\mathbf{{({chr(ord('`')+1+i)})}}$ {ticks[i_s]}",
                                                                                                xy=(-0.35,  0.95), 
                                                                                                xycoords='axes fraction',
                                                                                                horizontalalignment='left',
                                                                                                verticalalignment='bottom',
                                                                                                rotation=0,
                                                                                                fontsize=8.5) 

                                                    if False:
                                                        fig_test['ax'][i][0].annotate(ticks[i_s],
                                                                                                xy=(1.03, 0.5), xycoords='axes fraction',
                                                                                                horizontalalignment='left',
                                                                                                verticalalignment='center',
                                                                                                rotation=90,
                                                                                                fontsize=8.5)

                                                    for mod in pdict:
                                                        y_tick = np.array([(2*tt_test.shape[2]-(np.sum(pdict[mod]['index'][h,:][[0,-1]])+1)) for h in range(pdict[mod]['N'])]) / (2*tt_test.shape[2])

                                                        for h in range(pdict[mod]['N']):
                                                            if np.isin(mod, ['Ge', 'H']):
                                                                fig_test['ax'][i][0].annotate(f"$a_{{u}}$",
                                                                                                    xy=(-0.14, y_tick[h]), xycoords='axes fraction',
                                                                                                    horizontalalignment='right',
                                                                                                    verticalalignment='center',
                                                                                                    rotation=90,
                                                                                                    fontsize=fs)
                                                                fig_test['ax'][i][0].annotate(#f"$H{{\\left({pdict['H']['model'][-(1+h)]},{dist_symbol[hb_dist]}\\right)}}$",
                                                                                                f"${pdict[mod]['label'][h]}$",
                                                                                                    xy=(-0.2,  y_tick[h]), xycoords='axes fraction',
                                                                                                    horizontalalignment='right',
                                                                                                    verticalalignment='center',
                                                                                                    rotation=0,
                                                                                                    fontsize=fs) 
                                                            else:
                                                                fig_test['ax'][i][0].annotate(#f"$H{{\\left({pdict['H']['model'][-(1+h)]},{dist_symbol[hb_dist]}\\right)}}$",
                                                                                            f"${pdict[mod]['label'][h]}$",
                                                                                                xy=(-0.2,  y_tick[h]), xycoords='axes fraction',
                                                                                                horizontalalignment='right',
                                                                                                verticalalignment='center',
                                                                                                rotation=0,
                                                                                                fontsize=fs) 

                        


                                                    fig_test['ax'][i][-1].tick_params(labelright=False)


                                                for j in range(len(tails)):
                                                    fig_test['ax'][0][j].set_xlabel(f'{tails[j][0].upper()}{tails[j][1:]}-tail $\\{tails[j]}arrow}}$', fontsize=8.5)
                                                    fig_test['ax'][-1][j].set_xlabel(f'$a_{{q}}$', fontsize=fs)
                                                    fig_test['ax'][0][j].tick_params(labeltop=False)


                                                for i in range(max_i):
                                                    for j in range(len(tails)):
                                                        fig_test['ax'][i][j].set_xticks(a_u_ticks)
                                                        fig_test['ax'][i][j].tick_params(left=True, right=True, top=True, bottom=True,
                                                                                        labelleft=False, labelright=False, labeltop=False, labelbottom=False)
                                                    fig_test['ax'][i][0].tick_params(labelleft=True)
                                                for j in range(len(tails)):
                                                    fig_test['ax'][-1][j].tick_params(labelbottom=True)

                                            # fig_test['bar'].imshow(np.transpose([np.flip(np.linspace(0,1,1001))]), extent=[0,1,0,1], aspect=(8.5/7)*30,
                                            #                     cmap=cmap, norm=cmap_norm)
                                            # fig_test['bar'].set_ylabel(f'$p_{{{test_name}}}$', labelpad=-6)
                                                fig_test['bar'].imshow(np.transpose([np.flip(np.linspace(0,1,1001))]), extent=[0,1,0,1], aspect=1.2*(8.5/7)*30,
                                                            cmap=cmap, norm=cmap_norm)
                                                fig_test['bar'].set_ylabel(f'$p_{{\mathrm{{ZMD}}}}$', labelpad=-6, fontsize=fs)
                                                fig_test['bar'].set_yticks(bar_ticks)
                                                fig_test['bar'].set_yticklabels(bar_ticks, fontsize=fs)


                                                    
                                                fig_test['fig'].set_size_inches(5.7,6.5)
                                                fig_test['fig'].tight_layout()
                                                

                                                fstmp.fig_export(fig_test, width=5.7, height=6.5, 
                                                           #postsubs=folder
                                                           )


                                        
                        
zmd_test_f = copy.deepcopy(tt_test)
zmd_test = 1 - 2*np.abs(zmd_test_f - 0.5)